In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
!pip install datasets
!pip install python-dotenv
# !pip install pycocotools

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 8.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 6.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 12.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 10.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 16.0 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


In [4]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline
import random
from dotenv import load_dotenv
import os
import datasets as ds
import json
import pandas as pd

load_dotenv()

False

In [ ]:
# Check if CUDA is available and set device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

Using device: cuda


In [5]:
# from huggingface_hub import login
# hf_token = "HF_Token"
# login(hf_token)

In [6]:
# model_name = "meta-llama/Llama-2-7b-hf"
# tokenizer = AutoTokenizer.from_pretrained(model_name)

# model = AutoModelForCausalLM.from_pretrained(
#     model_name,
#     torch_dtype=torch.float16 if torch.cuda.is_available() else torch.float32,
#     device_map="auto" if torch.cuda.is_available() else None,
# )

In [7]:
# generator = pipeline(
#     "text-generation",
#     model=model,
#     tokenizer=tokenizer,
# )

In [8]:
# https://iambeginnerdeveloper.tistory.com/207
# Download COCO annotations
!wget http://images.cocodataset.org/annotations/annotations_trainval2017.zip

# Unzip the annotations
!unzip annotations_trainval2017.zip -d ./annotations

--2024-12-20 02:57:29--  http://images.cocodataset.org/annotations/annotations_trainval2017.zip
Resolving images.cocodataset.org (images.cocodataset.org)... 3.5.28.213, 3.5.29.216, 52.217.113.33, ...
Connecting to images.cocodataset.org (images.cocodataset.org)|3.5.28.213|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 252907541 (241M) [application/zip]
Saving to: ‘annotations_trainval2017.zip’

annotations_trainva 100%[===================>] 241.19M  20.0MB/s    in 13s     

2024-12-20 02:57:43 (18.0 MB/s) - ‘annotations_trainval2017.zip’ saved [252907541/252907541]

Archive:  annotations_trainval2017.zip
  inflating: ./annotations/annotations/instances_train2017.json  
  inflating: ./annotations/annotations/instances_val2017.json  
  inflating: ./annotations/annotations/captions_train2017.json  
  inflating: ./annotations/annotations/captions_val2017.json  
  inflating: ./annotations/annotations/person_keypoints_train2017.json  
  inflating: ./annotations/anno

In [10]:
# Path to the extracted captions file
ANNOTATION_FILE = "/content/annotations/annotations/captions_train2017.json"

# Load the JSON file
with open(ANNOTATION_FILE, "r") as f:
    data = json.load(f)

# Extract captions from the annotations
annotations = data['annotations']  # List of annotations

# Create a list of image-caption pairs
img_cap_pairs = []
for item in annotations:
    image_id = item['image_id']
    caption = item['caption']
    img_cap_pairs.append((image_id, caption))

# Convert to a DataFrame for easy handling
captions_df = pd.DataFrame(img_cap_pairs, columns=["image_id", "caption"])

# Randomly sample 500 captions
captions_sample = captions_df.sample(500, random_state=42)

# Print the 500 random captions
print("Random 500 Image Captions:")
for idx, row in captions_sample.iterrows():
    print(f"{idx + 1}: Image ID: {row['image_id']} | Caption: {row['caption']}")

Random 500 Image Captions:
264067: Image ID: 449919 | Caption: Two young men sitting on a bench and a lady standing next to them
31868: Image ID: 19783 | Caption: Seagulls in flight with a person feeding one with a lighthouse in the distance.
279035: Image ID: 574275 | Caption: Three spotted furry animals gazing in a field.
258964: Image ID: 184679 | Caption: A white truck with a lamb sitting on it's flatbed.
91525: Image ID: 468736 | Caption: A plate of pasta with meat and broccoli together
222235: Image ID: 310390 | Caption: A Dalmatian walks through a doorway leading outside.
190164: Image ID: 464888 | Caption: A group of people waiting to walk across street as people on bicycles ride by.
225334: Image ID: 171235 | Caption: A family of giraffes seen under trees across the way
15522: Image ID: 1803 | Caption: A bathroom with a blue tiled floor and a odd shapped toilet.
516361: Image ID: 425441 | Caption: a group of people sits on a couch posing 
562005: Image ID: 187400 | Caption: A 